<a href="https://colab.research.google.com/github/TomJiX/ML_start/blob/main/NN_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Package import

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time
import numpy as np
import gc
import os

Variable initialization

In [ ]:
X = np.load("_")
y= np.load("_")
SAVE_MODEL_NAME="_"

if os.path.isfile(SAVE_MODEL_NAME+".model"):
    SAVE_MODEL_NAME+=str(time.time())
    
X = X/255.0

# dense_layers = [0, 1, 2]
# layer_sizes = [32, 64, 128]
# conv_layers = [1, 2, 3]

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

EPOCHS=20

Choose the type of NN

In [ ]:
def general_fit():
    for dense_layer in dense_layers:
        for layer_size in layer_sizes:
            for conv_layer in conv_layers:
                NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
                print(NAME)

                model = Sequential()

                model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for l in range(conv_layer-1):
                    model.add(Conv2D(layer_size, (3, 3)))
                    model.add(Activation('relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())

                for _ in range(dense_layer):
                    model.add(Dense(layer_size))
                    model.add(Activation('relu'))

                model.add(Dense(1))
                model.add(Activation('sigmoid'))

                tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

                model.compile(loss='binary_crossentropy',
                            optimizer='adam',
                            metrics=['accuracy'],
                            )

                model.fit(X, y,
                        batch_size=32,
                        epochs=EPOCHS,
                        validation_split=0.3,
                        callbacks=[tensorboard])
                gc.collect()
    model.save(SAVE_MODEL_NAME+".model")


def particular_fit():
    NAME = "3conv32-64-128-1-dense"+str(time.time())
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X.shape[1:]))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes


    tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.fit(X, y,
                        batch_size=32,
                        epochs=EPOCHS,
                        validation_split=0.3,
                        callbacks=[tensorboard])

    

    gc.collect()
    model.save(SAVE_MODEL_NAME+".model")

Compute the models

In [ ]:
#particular_fit()
#general_fit()